# Lane keeping Prediction

In this notebook, we use a pre trained neural network from keras and use that to make on line predictions with our vehicle. Note that this notebook must be run on the vehicle. 

## Imports

In [1]:
%%capture
from tensorflow.train import AdamOptimizer
from tensorflow.keras.models import model_from_json
from PIL import Image
import numpy as np
import picamera
import easygopigo3 as easy
import time
import cv2
from utils import combinedThreshBinaryImg
from utils import absSobelThresh

## Load the trained model

In [2]:
json_file = open('final_trained_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("final_trained_model.h5")
loaded_model.compile(loss = 'mse', optimizer = AdamOptimizer())

## Predicting on New Pictures

#### Warm up the camera

In [ ]:
#functions to drive vehicles
my_gpg3 = easy.EasyGoPiGo3()
camera = picamera.PiCamera()

In [5]:
camera.resolution = (128, 128)
time.sleep(2)
motors = {'right':2, 'left':1}

#### Predict

In [ ]:
for i in range(100000):
    camera.capture('test.jpg', use_video_port=True)
    image = Image.open('test.jpg')
    image = np.array(image)
    image = image[60:,]
    image = combinedThreshBinaryImg(image, threshX=(1, 255), 
                                                    threshY=(50, 255), 
                                                    threshColorS=(1,255), 
                                                    threshColorU=(250,250), 
                                                 threshColorR=(230,255))

    image = image.reshape((1,68,128,1))
    pred = loaded_model.predict(image)
    left_speed = pred[0][0][0]
    right_speed = pred[1][0][0]
    current_speed_right = my_gpg3.set_motor_dps(motors['right'],right_speed)
    current_speed_left = my_gpg3.set_motor_dps(motors['left'],left_speed)
my_gpg3.stop()

In [ ]:
my_gpg3.stop()